# Ejecta model

TARDIS requires a model of the homologously expanding ejecta in order to run a simulation. A model will include information like the velocity cell structure, abundances, density, and radiation field.
This information is stored in TARDIS [config](link)
A short description of Properties and Base Property
##  Model grid
Explanation of grid and why we need velocity
### Velocity
Explain
Example
### Density
Explain
Example
### Abundance
Explain
Example
### Radiation Field
Explain
Example
## Generic[Ejecta]Model
A more flexible implementation of the TARDIS ejecta model is the Generic[Ejecta]Model. 
What does Generic model is and what it does with data.

Explain what you can do here but not in TARDIS
Start model and different times using above properties 
Explain that more properties can be added as long as they use the basxi structure of BaseProperty
## TARDIS Ejecta Model
Explain what happens with v_boundary and that it can't change.
